In [1]:
import re

from utils.jaso_complition import join_jamos

CHOSEONG = {
    '000100': 'ㄱ', '100100': 'ㄴ', '010100': 'ㄷ', 
    '000010': 'ㄹ', '100010': 'ㅁ', '000110': 'ㅂ', '000001': 'ㅅ',
    '110110': 'ㅇ', '000101': 'ㅈ', '000011': 'ㅊ',
    '110100': 'ㅋ', '110010': 'ㅌ', '100110': 'ㅍ', '010110': 'ㅎ'
}

DOUBLE = {
    '000100': 'ㄲ', '010100': 'ㄸ', '000110': 'ㅃ', '000001': 'ㅆ', '000101': 'ㅉ'
}

SHORT_WORD = { # 뒤에 오는 자소(자음 혹은 모음)에 따라 약자로 Mapping될 수 있다. 초성입력시 뒤에 SHORTWORD내부에 있는지 체크하고 뒤에 있는 거까지 window로 묶어 초성+초성, 초성+종성-> ShortWord로 처리 
    '110101': ['ㄱ','ㅏ'], '100100': ['ㄴ','ㅏ'], '010100': ['ㄷ','ㅏ'], '100010': ['ㅁ','ㅏ'], '000110': ['ㅂ','ㅏ'], '111000': ['ㅅ','ㅏ'], '000101': ['ㅈ','ㅏ'],
    '110100': ['ㅋ','ㅏ'], '110010': ['ㅌ','ㅏ'], '100110': ['ㅍ','ㅏ'], '010110': ['ㅎ','ㅏ']
}
SHORT_DIRECT_MAPPING = {    
    '100111': ['ㅓ','ㄱ'], '011111': ['ㅓ','ㄴ'], '011110': ['ㅓ','ㄹ'], 
    '100001': ['ㅕ','ㄴ'], '110011': ['ㅕ','ㄹ'], '110111': ['ㅕ','ㅇ'], '101101': ['ㅗ','ㄱ'], '111011': ['ㅗ','ㄴ'], '111111': ['ㅗ','ㅇ'], '110110': ['ㅜ','ㄴ'], '111101': ['ㅜ','ㄹ'],
    '101011': ['ㅡ','ㄴ'], '011101': ['ㅡ','ㄹ'], '111110': ['ㅣ','ㄴ'], '000111': ['것']
}

JUNGSEONG = {
    '110001': 'ㅏ', '001110': 'ㅑ', '011100': 'ㅓ', '100011': 'ㅕ', '101001': 'ㅗ',
    '001101': 'ㅛ', '101100': 'ㅜ', '100101': 'ㅠ', '010101': 'ㅡ', '101010': 'ㅣ',
    '111010': 'ㅐ', '101110': 'ㅔ', '001100': 'ㅖ', '101111': 'ㅚ',
    '111001': 'ㅘ', '111100': 'ㅝ', '010111': 'ㅢ'
}
######## 
DOUBLE_JUNGSUNG = { # Double Jungsung에 포함된 자소 -> 뒤에 ㅣ (101010) 가 온다면 아래와 같이 매핑후 같이 처리
    '111001': 'ㅙ',
    '001110': 'ㅒ',
    '111100': 'ㅞ',
    '101100': 'ㅟ',
}

JONGSEONG = {
    '100000': 'ㄱ', '010010': 'ㄴ', '001010': 'ㄷ', '010000': 'ㄹ', '010001': 'ㅁ', '110000': 'ㅂ', '001000': 'ㅅ',
    '011011': 'ㅇ', '101000': 'ㅈ', '011000': 'ㅊ', '011010': 'ㅋ', '011001': 'ㅌ', '010011': 'ㅍ', '001011': 'ㅎ'
}

DOUBLE_JONGSUNG = {
    '100000100000': 'ㄲ', '100000001000' : 'ㄳ', '010010101000' : 'ㄵ', '010010001011' : 'ㄶ', '010000100000':'ㄺ', '010000010001':'ㄻ', '010000110000':'ㄼ', '010000001000':'ㄽ', '01000011001':'ㄾ', 
    '010000010011':'ㄿ', '010000001011':'ㅀ', '110000001000':'ㅄ', '001000001000':'ㅆ'
}

UNG = {
    '셩': '성', '쎵': '썽', '졍': '정', '쪙': '쩡', '쳥': '청'
}

# 초성, 중성, 종성의 유니코드 매핑
CHOSEONG_MAP = {'ㄱ': 0, 'ㄲ': 1, 'ㄴ': 2, 'ㄷ': 3, 'ㄸ': 4, 'ㄹ': 5, 'ㅁ': 6, 'ㅂ': 7, 'ㅃ': 8, 'ㅅ': 9, 'ㅆ': 10, 'ㅇ': 11, 'ㅈ': 12, 'ㅉ': 13, 'ㅊ': 14, 'ㅋ': 15, 'ㅌ': 16, 'ㅍ': 17, 'ㅎ': 18}
JUNGSEONG_MAP = {'ㅏ': 0, 'ㅐ': 1, 'ㅑ': 2, 'ㅒ': 3, 'ㅓ': 4, 'ㅔ': 5, 'ㅕ': 6, 'ㅖ': 7, 'ㅗ': 8, 'ㅘ': 9, 'ㅙ': 10, 'ㅚ': 11, 'ㅛ': 12, 'ㅜ': 13, 'ㅝ': 14, 'ㅞ': 15, 'ㅟ': 16, 'ㅠ': 17, 'ㅡ': 18, 'ㅢ': 19, 'ㅣ': 20}
JONGSEONG_MAP = {' ': 0, 'ㄱ': 1, 'ㄲ': 2, 'ㄳ': 3, 'ㄴ': 4, 'ㄵ': 5, 'ㄶ': 6, 'ㄷ': 7, 'ㄹ': 8, 'ㄺ': 9, 'ㄻ': 10, 'ㄼ': 11, 'ㄽ': 12, 'ㄾ': 13, 'ㄿ': 14, 'ㅀ': 15, 'ㅁ': 16, 'ㅂ': 17, 'ㅄ': 18, 'ㅅ': 19, 'ㅆ': 20, 'ㅇ': 21, 'ㅈ': 22, 'ㅊ': 23, 'ㅋ': 24, 'ㅌ': 25, 'ㅍ': 26, 'ㅎ': 27}



In [2]:
toilet = "100100 010001 000101 010110 111001 000101 011011 000001 101010 010000".split() # 남자화장실 (약어)

chicken_skewd = '010100 010000 100000 000001 000100 101001 000011 101010'.split() # 닭꼬치

pretty = '001100 000001 000110 101011 101010'.split() # 예쁜이



In [16]:
result = []
l = toilet
skip_next = False  # 다음 원소를 건너뛸지 여부를 저장하는 플래그
print(l)
for m in range(len(l)):
    if skip_next:
        skip_next = False  # 이전에 m+1을 건너뛰기로 했으면 이번 루프는 처리하지 않고 넘김
        continue

    if l[m] in CHOSEONG:
        if m + 1 < len(l) and l[m] == '000001' and l[m+1] in CHOSEONG:
            result.append(DOUBLE[l[m+1]])
            skip_next = True  # 다음 원소를 건너뛰기 위해 플래그 설정
        elif m + 1 < len(l) and l[m+1] in DOUBLE:  # 된소리(초성) 처리
            combined_s = l[m]  
            result.append(DOUBLE[combined_s])  
            skip_next = True  # 다음 원소를 건너뛰기 위해 플래그 설정
            
        elif m + 1 < len(l) and l[m] in SHORT_WORD and (l[m+1] in JONGSEONG or l[m+1] in CHOSEONG):  # 약어 처리
            short_word = l[m]
            result += SHORT_WORD[short_word]
        else:
            result.append(CHOSEONG[l[m]])

    elif l[m] in JUNGSEONG:
        if (0 < m - 1 or m == 0 )and l[m-1] not in CHOSEONG: # 'ㅇ' 이 초성으로 올경우 추가 처리
            result.append('ㅇ')
        
        if m + 1 < len(l) and l[m+1] == '111010':  # 이중 모음 체크
            result.append(DOUBLE_JUNGSUNG[l[m]])
            skip_next = True  # 다음 원소를 건너뛰기 위해 플래그 설정
        else:
            result.append(JUNGSEONG[l[m]])

    elif l[m] in JONGSEONG:
        if m + 1 < len(l) and l[m+1] in JONGSEONG:
            combined_e = l[m] + l[m+1]

            result.append(DOUBLE_JONGSUNG[combined_e])
            skip_next = True  # 다음 원소를 건너뛰기 위해 플래그 설정
        else:
            result.append(JONGSEONG[l[m]])

    elif l[m] in SHORT_DIRECT_MAPPING:
        result += SHORT_DIRECT_MAPPING[l[m]]

['100100', '010001', '000101', '010110', '111001', '000101', '011011', '000001', '101010', '010000']


In [12]:
from utils.binary_to_jaso import jaso2char
result = jaso2char(toilet) 

['100100', '010001', '000101', '010110', '111001', '000101', '011011', '000001', '101010', '010000']


In [13]:
result

['ㄴ', 'ㅏ', 'ㅁ', 'ㅈ', 'ㅏ', 'ㅎ', 'ㅘ', 'ㅈ', 'ㅏ', 'ㅇ', 'ㅅ', 'ㅣ', 'ㄹ']

In [8]:
result

['ㄷ', 'ㅏ', 'ㄺ', 'ㄲ', 'ㅗ', 'ㅊ', 'ㅣ']

In [9]:
from utils.jaso_complition import join_jamos

print(join_jamos(result))

닭꼬치
